# Caravan PCA - MA755

## Introduction

In this notebook we will use PCA (Principal Component Analysis) to convert a set of observations of possibly correlated variables into a set of uncorrelated values called principal components.

## Dataset Description

Each row of the dataset represents a group of customers. The variables that start with "M" provide demographic data and contain integers between `0` and `9` that represent ranges of percentages with `0` representing 0% and `9` representing 100%. 
The other integers represent ranges of about 13% each. 
See the documentation for details. 

The variables that start with "A" or "P" provide information about the customers in that postal code. 
They contain integers between `0` and `9` that represent ranges of counts for that variable.
See the documentation for details. 

The `CARAVAN` variable is the target variable.
It is also a _count_ variable (as above) recording the number of mobile home policies in that postal code.

## Load Libraries

In the following cell we load the appropriate libraries into the notebook, and check the version numbers.

In [8]:
import numpy   as np
import pandas  as pd
import sklearn as sk
from sklearn_pandas import DataFrameMapper
from sklearn_pandas import gen_features
import sklearn.preprocessing, sklearn.decomposition, \
       sklearn.linear_model,  sklearn.pipeline, \
       sklearn.metrics
np.__version__, pd.__version__, sk.__version__

Out[ 1 ]: ('1.11.1', '0.18.1', '0.18.1')

## Read Dataset

The following cell reads in the dataset and assigns it to the `caravan_df` variable.  We then run the `.head()` commande to verify the data looks as expected.

In [11]:
caravan_df = pd.read_csv('/dbfs/mnt/group-ma755/data/caravan-insurance-challenge.csv')
caravan_df.head()

Out[ 2 ]: 
 ORIGIN MOSTYPE MAANTHUI MGEMOMV MGEMLEEF MOSHOOFD MGODRK MGODPR \
0 train 33 1 3 2 8 0 5 
1 train 37 1 2 2 8 1 4 
2 train 37 1 2 2 8 0 4 
3 train 9 1 3 3 3 2 3 
4 train 40 1 4 2 10 1 4 

 MGODOV MGODGE ... APERSONG AGEZONG AWAOREG ABRAND AZEILPL \
0 1 3 ... 0 0 0 1 0 
1 1 4 ... 0 0 0 1 0 
2 2 4 ... 0 0 0 1 0 
3 2 4 ... 0 0 0 1 0 
4 1 4 ... 0 0 0 1 0 

 APLEZIER AFIETS AINBOED ABYSTAND CARAVAN 
0 0 0 0 0 0 
1 0 0 0 0 0 
2 0 0 0 0 0 
3 0 0 0 0 0 
4 0 0 0 0 0 

[5 rows x 87 columns]

The target variable `CARAVAN` should not be considered in the process of building model so that our model will not bring bias. Therefore, we remove the `CARAVAN` variable in the feature dataset.

In [13]:
Y = ['CARAVAN']
caravan_df_features=caravan_df.loc[:, ~caravan_df.columns.isin(Y)]
caravan_df_features

Out[ 28 ]: 
 ORIGIN MOSTYPE MAANTHUI MGEMOMV MGEMLEEF MOSHOOFD MGODRK MGODPR \
0 train 33 1 3 2 8 0 5 
1 train 37 1 2 2 8 1 4 
2 train 37 1 2 2 8 0 4 
3 train 9 1 3 3 3 2 3 
4 train 40 1 4 2 10 1 4 
5 train 23 1 2 1 5 0 5 
6 train 39 2 3 2 9 2 2 
7 train 33 1 2 3 8 0 7 
8 train 33 1 2 4 8 0 1 
9 train 11 2 3 3 3 3 5 
10 train 10 1 4 3 3 1 4 
11 train 9 1 3 3 3 1 3 
12 train 33 1 2 3 8 1 4 
13 train 41 1 3 3 10 0 5 
14 train 23 1 1 2 5 0 6 
15 train 33 1 2 3 8 0 7 
16 train 38 1 2 3 9 0 6 
17 train 22 2 3 3 5 0 5 
18 train 13 1 4 2 3 2 4 
19 train 31 1 2 4 7 0 2 
20 train 33 1 4 3 8 0 6 
21 train 33 2 3 3 8 0 4 
22 train 13 1 3 2 3 1 7 
23 train 34 2 3 2 8 0 7 
24 train 13 2 4 3 3 0 4 
25 train 33 1 3 3 8 0 6 
26 train 37 1 3 3 8 0 5 
27 train 40 1 3 3 10 0 3 
28 train 31 1 4 2 7 0 9 
29 train 33 2 2 3 8 0 7 
... ... ... ... ... ... ... ... ... 
9792 test 24 1 3 3 5 1 2 
9793 test 10 1 3 2 3 1 5 
9794 test 34 1 3 3 8 0 2 
9795 test 10 2 4 3 3 1 5 
9796 test 39 2 5 2 9 1 5 
9797 test 38 1 3 3 9 0 3 
9798 test 23 1 2 3 5 3 2 
9799 test 40 1 2 4 10 0 6 
9800 test 38 1 3 3 9 0 6 
9801 test 39 1 2 4 9 0 4 
9802 test 33 1 3 3 8 0 4 
9803 test 41 1 3 3 10 0 5 
9804 test 10 2 3 3 3 2 4 
9805 test 11 1 3 2 3 1 4 
9806 test 33 1 2 4 8 0 1 
9807 test 13 1 3 2 3 2 3 
9808 test 8 1 3 3 2 1 6 
9809 test 38 1 2 3 9 1 3 
9810 test 33 1 3 3 8 0 6 
9811 test 12 1 3 2 3 0 7 
9812 test 33 1 3 3 8 0 3 
9813 test 22 1 2 2 5 0 6 
9814 test 22 2 3 3 5 0 9 
9815 test 33 1 2 4 8 0 5 
9816 test 10 1 4 3 3 0 5 
9817 test 33 1 2 4 8 0 7 
9818 test 24 1 2 3 5 1 5 
9819 test 36 1 2 3 8 1 5 
9820 test 33 1 3 3 8 1 4 
9821 test 8 1 2 3 2 4 3 

 MGODOV MGODGE ... ALEVEN APERSONG AGEZONG AWAOREG ABRAND \
0 1 3 ... 0 0 0 0 1 
1 1 4 ... 0 0 0 0 1 
2 2 4 ... 0 0 0 0 1 
3 2 4 ... 0 0 0 0 1 
4 1 4 ... 0 0 0 0 1 
5 0 5 ... 0 0 0 0 0 
6 0 5 ... 0 0 0 0 0 
7 0 2 ... 0 0 0 0 0 
8 3 6 ... 0 0 0 0 0 
9 0 2 ... 0 0 0 0 1 
10 1 4 ... 0 0 0 0 0 
11 2 4 ... 0 0 0 0 1 
12 1 4 ... 0 0 0 0 0 
13 0 4 ... 0 0 0 0 0 
14 1 2 ... 0 0 0 0 1 
15 0 2 ... 0 0 0 0 1 
16 0 3 ... 0 0 0 0 0 
17 0 4 ... 0 0 0 0 1 
18 0 3 ... 0 0 0 0 1 
19 0 7 ... 0 0 0 0 0 
20 0 3 ... 0 0 0 0 0 
21 2 3 ... 0 0 0 0 1 
22 0 2 ... 0 0 0 0 1 
23 0 2 ... 0 0 0 0 0 
24 2 4 ... 0 0 0 0 0 
25 1 2 ... 0 0 0 0 1 
26 0 4 ... 0 0 0 0 0 
27 0 6 ... 0 0 0 0 1 
28 0 0 ... 0 0 0 0 0 
29 1 2 ... 0 0 0 0 1 
... ... ... ... ... ... ... ... ... 
9792 1 6 ... 0 0 0 0 0 
9793 1 3 ... 0 0 0 0 1 
9794 2 5 ... 0 0 0 0 1 
9795 1 4 ... 1 0 0 0 1 
9796 0 3 ... 0 0 0 0 1 
9797 0 6 ... 0 0 0 0 1 
9798 1 3 ... 0 0 0 0 0 
9799 0 3 ... 0 0 0 0 1 
9800 2 3 ... 0 0 0 0 0 
9801 1 5 ... 0 0 0 0 0 
9802 1 5 ... 0 0 0 0 1 
9803 1 3 ... 0 0 0 0 1 
9804 1 3 ... 0 0 0 0 0 
9805 0 5 ... 2 0 0 0 1 
9806 3 6 ... 0 0 0 0 0 
9807 1 5 ... 0 0 0 0 1 
9808 1 3 ... 0 0 0 0 1 
9809 1 5 ... 0 0 0 0 0 
9810 1 2 ... 2 0 0 0 1 
9811 2 0 ... 0 0 0 0 1 
9812 0 6 ... 0 0 0 0 0 
9813 1 2 ... 0 0 0 0 0 
9814 0 0 ... 0 0 0 0 1 
9815 2 2 ... 0 0 0 0 1 
9816 2 3 ... 0 0 0 0 0 
9817 2 0 ... 0 0 0 0 1 
9818 1 3 ... 1 0 0 0 1 
9819 1 3 ... 0 0 0 0 1 
9820 2 3 ... 0 0 0 0 0 
9821 0 3 ... 0 0 0 0 1 

 AZEILPL APLEZIER AFIETS AINBOED ABYSTAND 
0 0 0 0 0 0 
1 0 0 0 0 0 
2 0 0 0 0 0 
3 0 0 0 0 0 
4 0 0 0 0 0 
5 0 0 0 0 0 
6 0 0 0 0 0 
7 0 0 0 0 0 
8 0 0 0 0 0 
9 0 0 0 0 0 
10 0 0 0 0 0 
11 0 0 0 0 0 
12 0 0 0 0 0 
13 0 0 0 0 0 
14 0 0 0 0 0 
15 0 0 0 0 0 
16 0 0 0 0 0 
17 0 0 0 0 0 
18 0 0 0 0 0 
19 0 0 0 0 0 
20 0 0 0 0 0 
21 0 0 0 0 0 
22 0 0 0 0 0 
23 0 0 0 0 0 
24 0 0 1 0 0 
25 0 0 0 0 0 
26 0 0 0 0 0 
27 0 0 0 0 0 
28 0 0 0 0 0 
29 0 0 0 0 0 
... ... ... ... ... ... 
9792 0 0 0 0 0 
9793 0 0 0 0 0 
9794 0 0 0 0 0 
9795 0 0 0 0 0 
9796 0 0 0 0 0 
9797 0 0 0 0 0 
9798 0 0 0 0 0 
9799 0 0 0 0 0 
9800 0 0 0 0 0 
9801 0 0 0 0 0 
9802 0 0 0 0 0 
9803 0 0 0 0 0 
9804 0 0 0 0 0 
9805 0 0 0 0 0 
9806 0 0 0 0 0 
9807 0 0 0 0 1 
9808 0 0 0 0 0 
9809 0 0 0 0 0 
9810 0 0 0 1 0 
9811 0 0 0 0 0 
9812 0 0 0 0 0 
9813 0 0 0 0 0 
9814 0 0 0 0 0 
9815 0 0 0 0 0 
9816 0 0 0 0 0 
9817 0 0 0 0 0 


In [14]:
train=caravan_df_features[caravan_df_features.ORIGIN=="train"]
train

Out[ 30 ]: 
 ORIGIN MOSTYPE MAANTHUI MGEMOMV MGEMLEEF MOSHOOFD MGODRK MGODPR \
0 train 33 1 3 2 8 0 5 
1 train 37 1 2 2 8 1 4 
2 train 37 1 2 2 8 0 4 
3 train 9 1 3 3 3 2 3 
4 train 40 1 4 2 10 1 4 
5 train 23 1 2 1 5 0 5 
6 train 39 2 3 2 9 2 2 
7 train 33 1 2 3 8 0 7 
8 train 33 1 2 4 8 0 1 
9 train 11 2 3 3 3 3 5 
10 train 10 1 4 3 3 1 4 
11 train 9 1 3 3 3 1 3 
12 train 33 1 2 3 8 1 4 
13 train 41 1 3 3 10 0 5 
14 train 23 1 1 2 5 0 6 
15 train 33 1 2 3 8 0 7 
16 train 38 1 2 3 9 0 6 
17 train 22 2 3 3 5 0 5 
18 train 13 1 4 2 3 2 4 
19 train 31 1 2 4 7 0 2 
20 train 33 1 4 3 8 0 6 
21 train 33 2 3 3 8 0 4 
22 train 13 1 3 2 3 1 7 
23 train 34 2 3 2 8 0 7 
24 train 13 2 4 3 3 0 4 
25 train 33 1 3 3 8 0 6 
26 train 37 1 3 3 8 0 5 
27 train 40 1 3 3 10 0 3 
28 train 31 1 4 2 7 0 9 
29 train 33 2 2 3 8 0 7 
... ... ... ... ... ... ... ... ... 
5792 train 13 1 3 3 3 1 5 
5793 train 30 1 3 3 7 0 4 
5794 train 33 1 3 2 8 0 2 
5795 train 22 1 2 2 5 0 3 
5796 train 39 1 4 2 9 0 5 
5797 train 8 1 4 3 2 0 6 
5798 train 27 1 1 6 6 1 5 
5799 train 9 1 3 3 3 0 5 
5800 train 31 1 3 3 7 2 3 
5801 train 3 1 2 4 1 0 6 
5802 train 3 1 2 4 1 1 4 
5803 train 36 1 3 3 8 0 6 
5804 train 23 1 2 2 5 2 4 
5805 train 33 1 3 3 8 0 6 
5806 train 38 2 3 2 9 0 6 
5807 train 38 2 3 4 9 0 9 
5808 train 31 1 3 2 7 2 2 
5809 train 35 1 2 3 8 0 7 
5810 train 33 1 3 2 8 0 4 
5811 train 39 1 3 2 9 1 5 
5812 train 40 1 3 4 10 0 7 
5813 train 3 1 2 4 1 1 6 
5814 train 1 1 3 3 1 1 4 
5815 train 13 1 2 3 3 0 2 
5816 train 3 1 2 3 1 0 6 
5817 train 36 1 1 2 8 0 6 
5818 train 35 1 4 4 8 1 4 
5819 train 33 1 3 4 8 0 6 
5820 train 34 1 3 2 8 0 7 
5821 train 33 1 3 3 8 0 6 

 MGODOV MGODGE ... ALEVEN APERSONG AGEZONG AWAOREG ABRAND \
0 1 3 ... 0 0 0 0 1 
1 1 4 ... 0 0 0 0 1 
2 2 4 ... 0 0 0 0 1 
3 2 4 ... 0 0 0 0 1 
4 1 4 ... 0 0 0 0 1 
5 0 5 ... 0 0 0 0 0 
6 0 5 ... 0 0 0 0 0 
7 0 2 ... 0 0 0 0 0 
8 3 6 ... 0 0 0 0 0 
9 0 2 ... 0 0 0 0 1 
10 1 4 ... 0 0 0 0 0 
11 2 4 ... 0 0 0 0 1 
12 1 4 ... 0 0 0 0 0 
13 0 4 ... 0 0 0 0 0 
14 1 2 ... 0 0 0 0 1 
15 0 2 ... 0 0 0 0 1 
16 0 3 ... 0 0 0 0 0 
17 0 4 ... 0 0 0 0 1 
18 0 3 ... 0 0 0 0 1 
19 0 7 ... 0 0 0 0 0 
20 0 3 ... 0 0 0 0 0 
21 2 3 ... 0 0 0 0 1 
22 0 2 ... 0 0 0 0 1 
23 0 2 ... 0 0 0 0 0 
24 2 4 ... 0 0 0 0 0 
25 1 2 ... 0 0 0 0 1 
26 0 4 ... 0 0 0 0 0 
27 0 6 ... 0 0 0 0 1 
28 0 0 ... 0 0 0 0 0 
29 1 2 ... 0 0 0 0 1 
... ... ... ... ... ... ... ... ... 
5792 1 3 ... 0 0 0 0 1 
5793 1 4 ... 0 0 0 0 0 
5794 4 4 ... 0 0 0 0 1 
5795 1 6 ... 0 0 0 0 0 
5796 1 4 ... 0 0 0 0 0 
5797 1 2 ... 0 0 0 0 1 
5798 2 3 ... 0 0 0 0 1 
5799 0 4 ... 0 0 0 0 1 
5800 0 5 ... 1 0 0 0 1 
5801 1 3 ... 0 0 0 0 1 
5802 1 4 ... 0 0 0 0 0 
5803 0 3 ... 0 0 0 0 0 
5804 1 3 ... 0 0 0 0 1 
5805 1 2 ... 0 0 0 0 0 
5806 1 2 ... 0 0 0 0 1 
5807 0 0 ... 0 0 0 0 1 
5808 0 5 ... 2 0 0 0 0 
5809 0 2 ... 0 0 0 0 0 
5810 0 5 ... 0 0 0 0 0 
5811 1 3 ... 0 0 0 0 1 
5812 0 2 ... 0 0 0 0 2 
5813 1 2 ... 0 0 0 0 1 
5814 2 3 ... 0 0 0 0 1 
5815 0 7 ... 0 0 0 0 1 
5816 0 3 ... 0 0 0 0 0 
5817 1 2 ... 2 0 0 0 1 
5818 1 4 ... 0 0 0 0 1 
5819 0 3 ... 0 0 0 0 1 
5820 0 2 ... 0 0 0 0 0 
5821 1 2 ... 0 0 0 0 0 

 AZEILPL APLEZIER AFIETS AINBOED ABYSTAND 
0 0 0 0 0 0 
1 0 0 0 0 0 
2 0 0 0 0 0 
3 0 0 0 0 0 
4 0 0 0 0 0 
5 0 0 0 0 0 
6 0 0 0 0 0 
7 0 0 0 0 0 
8 0 0 0 0 0 
9 0 0 0 0 0 
10 0 0 0 0 0 
11 0 0 0 0 0 
12 0 0 0 0 0 
13 0 0 0 0 0 
14 0 0 0 0 0 
15 0 0 0 0 0 
16 0 0 0 0 0 
17 0 0 0 0 0 
18 0 0 0 0 0 
19 0 0 0 0 0 
20 0 0 0 0 0 
21 0 0 0 0 0 
22 0 0 0 0 0 
23 0 0 0 0 0 
24 0 0 1 0 0 
25 0 0 0 0 0 
26 0 0 0 0 0 
27 0 0 0 0 0 
28 0 0 0 0 0 
29 0 0 0 0 0 
... ... ... ... ... ... 
5792 0 0 0 0 0 
5793 0 0 0 0 0 
5794 0 0 0 0 0 
5795 0 0 0 0 0 
5796 0 0 0 0 0 
5797 0 0 0 0 0 
5798 0 0 0 0 0 
5799 0 0 0 0 0 
5800 0 0 0 0 0 
5801 0 0 0 0 0 
5802 0 0 0 0 0 
5803 0 0 0 0 0 
5804 0 0 0 0 0 
5805 0 0 0 0 0 
5806 0 0 0 0 0 
5807 0 0 0 0 0 
5808 0 0 0 0 0 
5809 0 0 0 0 0 
5810 0 0 0 0 0 
5811 0 0 0 0 0 
5812 0 0 0 0 0 
5813 0 0 0 0 0 
5814 0 0 0 0 0 
5815 0 0 0 0 0 
5816 0 0

In the next 2 cells we run the `.info` commmand on the `caravan_df_features` variable to make sure the data structure is as expected, and also to find the total number of rows in the dataframe, which we then assign to the `caravan_rows` variable.

In [16]:
caravan_df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9822 entries, 0 to 9821
Data columns (total 85 columns):
MOSTYPE 9822 non-null int64
MAANTHUI 9822 non-null int64
MGEMOMV 9822 non-null int64
MGEMLEEF 9822 non-null int64
MOSHOOFD 9822 non-null int64
MGODRK 9822 non-null int64
MGODPR 9822 non-null int64
MGODOV 9822 non-null int64
MGODGE 9822 non-null int64
MRELGE 9822 non-null int64
MRELSA 9822 non-null int64
MRELOV 9822 non-null int64
MFALLEEN 9822 non-null int64
MFGEKIND 9822 non-null int64
MFWEKIND 9822 non-null int64
MOPLHOOG 9822 non-null int64
MOPLMIDD 9822 non-null int64
MOPLLAAG 9822 non-null int64
MBERHOOG 9822 non-null int64
MBERZELF 9822 non-null int64
MBERBOER 9822 non-null int64
MBERMIDD 9822 non-null int64
MBERARBG 9822 non-null int64
MBERARBO 9822 non-null int64
MSKA 9822 non-null int64
MSKB1 9822 non-null int64
MSKB2 9822 non-null int64
MSKC 9822 non-null int64
MSKD 9822 non-null int64
MHHUUR 9822 non-null int64
MHKOOP 9822 non-null int64
MAUT1 9822 non-null int64
MAUT2 9822 non-null int64
MAUT0 9822 non-null int64
MZFONDS 9822 non-null int64
MZPART 9822 non-null int64
MINKM30 9822 non-null int64
MINK3045 9822 non-null int64
MINK4575 9822 non-null int64
MINK7512 9822 non-null int64
MINK123M 9822 non-null int64
MINKGEM 9822 non-null int64
MKOOPKLA 9822 non-null int64
PWAPART 9822 non-null int64
PWABEDR 9822 non-null int64
PWALAND 9822 non-null int64
PPERSAUT 9822 non-null int64
PBESAUT 9822 non-null int64
PMOTSCO 9822 non-null int64
PVRAAUT 9822 non-null int64
PAANHANG 9822 non-null int64
PTRACTOR 9822 non-null int64
PWERKT 9822 non-null int64
PBROM 9822 non-null int64
PLEVEN 9822 non-null int64
PPERSONG 9822 non-null int64
PGEZONG 9822 non-null int64
PWAOREG 9822 non-null int64
PBRAND 9822 non-null int64
PZEILPL 9822 non-null int64
PPLEZIER 9822 non-null int64
PFIETS 9822 non-null int64
PINBOED 9822 non-null int64
PBYSTAND 9822 non-null int64
AWAPART 9822 non-null int64
AWABEDR 9822 non-null int64
AWALAND 9822 non-null int64
APERSAUT 9822 non-null int64
ABESAUT 9822 non-null int64
AMOTSCO 9822 non-null int64
AVRAAUT 9822 non-null int64
AAANHANG 9822 non-null int64
ATRACTOR 9822 non-null int64
AWERKT 9822 non-null int64
ABROM 9822 non-null int64
ALEVEN 9822 non-null int64
APERSONG 9822 non-null int64
AGEZONG 9822 non-null int64
AWAOREG 9822 non-null int64
ABRAND 9822 non-null int64
AZEILPL 9822 non-null int64
APLEZIER 9822 non-null int64
AFIETS 9822 non-null int64
AINBOED 9822 non-null int64
ABYSTAND 9822 non-null int64
dtypes: int64(85)
memory usage: 6.4 MB

In this cell we create the lists we will use in order to binarize the dataframe.  We create the variable `feature_list`, and use this variable to create two lists, `binarizer_list` and `scaler_list`.  The `binarizer_list` will include all the variables we want to binarize (make either a 1 or 0).  The `scalar_list` will include all the other variables in our dataframe. We accomplish this in two steps.  For the `binarizer_list`, we include any columns for which the name is present in `feature_list`.  For the `scalar_list`, we include any columns for which the name is not present in `feature_list`.  We then print out both lists to verify that the code has worked properly.

In [18]:
feature_list = ['MOSHOOFD','MOSTYPE']
binarizer_list = [[name] for name in list(caravan_df_features.columns) if name     in feature_list]
scaler_list    = [[name] for name in list(caravan_df_features.columns) if name not in feature_list]
binarizer_list, scaler_list

Out[ 18 ]: 
([['MOSTYPE'], ['MOSHOOFD']],
 [['MAANTHUI'],
 ['MGEMOMV'],
 ['MGEMLEEF'],
 ['MGODRK'],
 ['MGODPR'],
 ['MGODOV'],
 ['MGODGE'],
 ['MRELGE'],
 ['MRELSA'],
 ['MRELOV'],
 ['MFALLEEN'],
 ['MFGEKIND'],
 ['MFWEKIND'],
 ['MOPLHOOG'],
 ['MOPLMIDD'],
 ['MOPLLAAG'],
 ['MBERHOOG'],
 ['MBERZELF'],
 ['MBERBOER'],
 ['MBERMIDD'],
 ['MBERARBG'],
 ['MBERARBO'],
 ['MSKA'],
 ['MSKB1'],
 ['MSKB2'],
 ['MSKC'],
 ['MSKD'],
 ['MHHUUR'],
 ['MHKOOP'],
 ['MAUT1'],
 ['MAUT2'],
 ['MAUT0'],
 ['MZFONDS'],
 ['MZPART'],
 ['MINKM30'],
 ['MINK3045'],
 ['MINK4575'],
 ['MINK7512'],
 ['MINK123M'],
 ['MINKGEM'],
 ['MKOOPKLA'],
 ['PWAPART'],
 ['PWABEDR'],
 ['PWALAND'],
 ['PPERSAUT'],
 ['PBESAUT'],
 ['PMOTSCO'],
 ['PVRAAUT'],
 ['PAANHANG'],
 ['PTRACTOR'],
 ['PWERKT'],
 ['PBROM'],
 ['PLEVEN'],
 ['PPERSONG'],
 ['PGEZONG'],
 ['PWAOREG'],
 ['PBRAND'],
 ['PZEILPL'],
 ['PPLEZIER'],
 ['PFIETS'],
 ['PINBOED'],
 ['PBYSTAND'],
 ['AWAPART'],
 ['AWABEDR'],
 ['AWALAND'],
 ['APERSAUT'],
 ['ABESAUT'],
 ['AMOTSCO'],
 ['AVRAAUT'],
 ['AAANHANG'],
 ['ATRACTOR'],
 ['AWERKT'],
 ['ABROM'],
 ['ALEVEN'],
 ['APERSONG'],
 ['AGEZONG'],
 ['AWAOREG'],
 ['ABRAND'],
 ['AZEILPL'],
 ['APLEZIER'],
 ['AFIETS'],
 ['AINBOED'],
 ['ABYSTAND']])

Next we create the pipeline, passing both the `binarizer_list` and the `scalar_list` into the pipeline using `FeatureUnion`, and transform our dataframe into a numpy array.   We then use .fit_transform() command on the dataframe to verify that the mapper behaves as expected.  The `np.round` command simply rounds the values to a number of decimals, given the `decimals` criteria at the end of the command.

In [20]:
pipe = sklearn.pipeline.FeatureUnion([('binarizer', DataFrameMapper(gen_features(columns=binarizer_list,classes=[{'class': sklearn.preprocessing.LabelBinarizer}]))), 
                                      ('scaler', DataFrameMapper(gen_features(columns=scaler_list,classes=[{'class': sklearn.preprocessing.MinMaxScaler}])))])
Binarized_Caravan = np.round(pipe.fit_transform(caravan_df_features), decimals=2)

/databricks/python/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
 warnings.warn(msg, _DataConversionWarning)

## PCA Analysis

The following cell defines the `MyTransformer` class that we will use the in pipeline.  `MyTransformer` subtracts the column mean from each value in the column.

In [23]:
from sklearn.base import BaseEstimator, TransformerMixin
class MyTransformer(BaseEstimator, TransformerMixin):
  def __init__(self,my_var=False):
    self.my_var = my_var
  def fit(self, X, y=None):
    self.col_mean_ = np.mean(X,axis=0)
    return self
  def transform(self, X):
    ret_df = X - self.col_mean_
    return ret_df

The following cell defines the `PCATransformer` class that we will use the in pipeline.  `PCATransformer` creates, then sorts, the eigenvalues and eigenvectors.  Then creates a covariance matrix using the dot product of the now mean adjusted dataframe and the eigenvectors.

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
class PCATransformer(BaseEstimator, TransformerMixin):
  def __init__(self,my_var=False):
    self.my_var = my_var
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    eig_val, eig_vec = np.linalg.eig(X.T.dot(X)/len(X))
    idx = eig_val.argsort()[::-1]
    explained_variance_eigenValues = eig_val[idx]
    explained_variance_eigenVectors = eig_vec[:,idx]
    AQ_cov = np.cov(m=X.dot(explained_variance_eigenVectors),rowvar=False,bias=True)
    final_data=X.dot(explained_variance_eigenVectors)
    return AQ_cov, final_data

Here we now create the pipeline using the pipe we created previously that binarizes the dataframe.  The pipeline then passes this first into the `MyTransformer` class, then into the `PCATransformer` class created in the cells directly above.  We then pass the `caravan_df` dataframe into the pipeline using the `np.round` command in order to round the results to 4 decimal places.

In [27]:
from sklearn import pipeline
from sklearn import preprocessing

ml_pipeline=pipeline.Pipeline([('Binarizer', pipe),
                              ('std', MyTransformer()),
                              ('pca', PCATransformer())
                              ])
AQ_cov, final_data =ml_pipeline.fit_transform(caravan_df_features)
B=np.round(AQ_cov.astype('float64'), decimals=4)
B

/databricks/python/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
 warnings.warn(msg, _DataConversionWarning)
/tmp/1519756233645-0/PythonShell.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
 import six.moves.queue
 Out[ 24 ]: 
array([[ 0.5248, 0. , 0. , ..., 0. , -0. , -0. ],
 [ 0. , 0.3657, 0. , ..., -0. , 0. , 0. ],
 [ 0. , 0. , 0.2078, ..., -0. , 0. , -0. ],
 ..., 
 [ 0. , -0. , -0. , ..., 0. , 0. , -0. ],
 [-0. , 0. , 0. , ..., 0. , 0. , -0. ],
 [-0. , 0. , -0. , ..., -0. , -0. , 0. ]])

We put the value of target variable `CARAVAN` as label for our outcome. To visualize our dimensionally reduced features, we make the 86 features appear in the 2 mixed features using the eigen vectors matrix we generated during PCA transform process. The plot shows how observations are distributed in a 2-dimensional way. The red dots represent having purchased a caravan policy and the green dots represent those who have not purchased a policy.

In [29]:
import matplotlib.pyplot as plt
label1=caravan_df[caravan_df.CARAVAN==1]
label0=caravan_df[caravan_df.CARAVAN==0]
plt.scatter(final_data[label0.index,0], final_data[label0.index,1], alpha=0.5, color='green')
display(plt.show())
plt.scatter(final_data[label1.index,0], final_data[label1.index,1], alpha=0.5, color='red')
display(plt.show())
plt.title('Data transformed with 2 eigenvectors')

### Conclusion

In this notebook we create readin the `caravan` dataset and assign it to a dataframe labeled `caravan_df`.  We then binarize the data using the `LabelBinarizer` function for categorical variables, and `MinMaxScaler` for numerical variables.  We created a class called `PCATransformer` which calculates the eigenvalues & eigenvectors for the dataset, sorts them, then creates a covariance matrix based on the dataframe and the eigenvectors. It also returns a dot product of dataframe and eigenvectors.  Finally, we create a pipeline (shown below) that passes a dataframe into the binarizer function called `pipe`, which in turn is passed into the class `MyTransformer`, which subtracts the column mean from each value in that column.  Finally, this is passed into `PCATransformer`.   

`ml_pipeline=pipeline.Pipeline([('Binarizer', pipe),
                              ('std', MyTransformer()),
                              ('pca', PCATransformer())
                              ])`
                              
As expected, the pipeline returns a diagonal matrix where the only non-zero values reside in the diagonal of the matrix (similar to an identity matrix).  The values decrease as we move from left to right, which is also as expected since the eigenvectors were sorted in a descending order.